In [1]:
import MGTomo.model as mgmodel
import numpy as np
import astra
import MGTomo.tomoprojection as mgproj
from MGTomo.utils import myexp, mylog, mydiv
import MGTomo.Yfunctions as fcts

import torch
from torch.func import grad

In [2]:
max_levels = 3

In [3]:
x_orig = np.random.randn(63, 63)
#x_torch = torch.from_numpy(x_orig)
x_torch = torch.ones(63,63)*0.2
x_torch.requires_grad = True

model = mgmodel.astra_model(63,{'mode' : 'line', 'num_angles' : 50, 'level_decrease' : 1})
fine_dim = model.dim
A = [mgproj.TomoTorch(model.proj_factory(fine_dim))]
b = [A[0](x_torch)]
level = {int(np.sqrt(A[0].shape[1])): 0}

for i in range(1,max_levels+1):
    corse_dim = model.reduce_dim(fine_dim)
    A.append(mgproj.TomoTorch(model.proj_factory(corse_dim)))
    b.append(torch.from_numpy(model.reduce_rhs(b[-1].detach().numpy(), fine_dim, corse_dim)))
    level.update({int(np.sqrt(A[i].shape[1])): i})
    fine_dim=corse_dim

In [4]:
print(level)

{63: 0, 31: 1, 15: 2, 7: 3}


In [5]:
for i in range(max_levels+1):
    assert b[i].shape[0]*b[i].shape[1] == A[i].shape[0], 'dimension mismatch'
    print(f'level {i}:', b[i].shape[0]*b[i].shape[1], A[i].shape[0], np.sqrt(A[i].shape[1]))

level 0: 3150 3150 63.0
level 1: 1550 1550 31.0
level 2: 750 750 15.0
level 3: 350 350 7.0


In [6]:
def R(y):
    x = y[1:-1:2, 1:-1:2]
    return x

In [7]:
def coarsen_fn(fh, x, y0, l):
    x0 = R(y0)
    x0.retain_grad()
    
    fH = lambda x: fcts.kl_distance(x, A[l], b[l])
    
    fhy0 = fh(y0)
    fhy0.backward()
    
    fHx0 = fH(x0)
    fHx0.backward()
    
    kappa = R(y0.grad) - x0.grad
    val = fH(x) + torch.sum(kappa * x)
    
    return val

In [8]:
fh = lambda x: fcts.kl_distance(x, A[0], b[0])

In [17]:
y0 = torch.ones(63, 63)
y0.requires_grad = True

x = torch.ones(31,31)*0.4

psi = lambda x: coarsen_fn(fh, x, y0, 1)
psi(x)

tensor(35329.7688, dtype=torch.float64)

In [ ]:
def MLO(fh, y, l=0):
    x = R(y)
    y0, x0 = y, R(y)
    psi = lambda x: coarsen_fn(fh, x, y0, l+1)
    
    for i in range(maxIter[l]):
        x = fcts.SMART(psi, x, tau)
        
    if l < max_levels:
        x = MLO(psi, x, l+1)
        
    d = P(x-x0)
    y = linesearch(fh, y0, d)
    
    for i in range(maxIter[l]):
        y = fcts.SMART(psi, y, tau)
    
    return y